In [1]:
%pip install "ray[tune]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 MB 1.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 3.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 2.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.54.2
    Uninstalling grpcio-1.54.2:
      Successfully uninstalled grpcio-1.54.2
Note: you may need to restart the kernel to use updated packages.


In [2]:
from ray import tune

def objective(config):  # ①
    score = config["a"] ** 2 + config["b"]
    return {"score": score}


search_space = {  # ②
    "a": tune.grid_search([0.001, 0.01, 0.1, 1.0]),
    "b": tune.choice([1, 2, 3]),
}

tuner = tune.Tuner(objective, param_space=search_space)  # ③

results = tuner.fit()
print(results.get_best_result(metric="score", mode="min").config)

2023-06-30 19:04:00,160	INFO worker.py:1636 -- Started a local Ray instance.
2023-06-30 19:04:04,432	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


Trial name,date,done,hostname,iterations_since_restore,node_ip,pid,score,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
objective_1e5c6_00002,2023-06-30_19-04-45,False,DESKTOP-UVK0SLT,1,172.27.82.99,5008,1.01,0.0314031,0.0314031,0.0314031,1688144685,1,1e5c6_00002


2023-06-30 19:04:46,436	INFO tune.py:1111 -- Total run time: 41.65 seconds (28.62 seconds for the tuning loop).


{'a': 0.001, 'b': 1}


In [1]:
%cd ..

/home/jakob/Bachelorarbeit/GNNDensityGradients


In [2]:
from pytorch_lightning.tuner import Tuner
from utils.visualization import fig_to_tensor
from glob import glob
import os.path
import pickle
import zstandard as zstd
import msgpack
import msgpack_numpy
msgpack_numpy.patch()

In [3]:
def transform_msgpack_file(filepath, transform):
    """
    Given a .zst file, applies the given transform to all stored simulation states and writes the result back to the file.
    """
    # TODO: Implement this to allow storing of preprocessed data
    # See https://github.com/isl-org/DeepLagrangianFluids/blob/d651c6fdf2aca3fac9abe3693b20981b191b4769/datasets/create_physics_records.py#L100

    decompressor = zstd.ZstdDecompressor()
    with open(filepath, 'rb') as f:
        content = msgpack.unpackb(decompressor.decompress(f.read()), raw=False)

    for i in range(1, len(content)):
        content[i]['box'] = content[0]['box']
        content[i]['box_normals'] = content[0]['box_normals']

    content = [transform(sample) for sample in content]

    for i in range(1, len(content)):
        content[i]['box'] = None
        content[i]['box_normals'] = None

    compressor = zstd.ZstdCompressor(level=22)
    with open(filepath, 'wb') as f:
        f.write(compressor.compress(msgpack.packb(content, use_bin_type=True)))

    return

In [3]:
from utils.transforms import *
import torch
from utils.sph import *
import os
path = 'datasets/data/dpi_dam_break/train/test.zst'
os.path.isfile(path)

False

In [5]:
transformations = [
    ToSample(device='cpu'),
    AddDensity(include_box=False),
    AddTemporalDensityGradient(include_box=False),
    NormalizeDensityData(),
    ToNumpy()
]

In [9]:
from torchvision.transforms import Compose
transform = Compose(transformations)

transform_msgpack_file(path, transform)

In [5]:
path = 'datasets/data/ours_default_data/train/sim_0001_00.msgpack.zst'

In [6]:
filepath = path

decompressor = zstd.ZstdDecompressor()
with open(filepath, 'rb') as f:
    content = msgpack.unpackb(decompressor.decompress(f.read()), raw=False)

content[1]

{'frame_id': 1,
 'scene_id': 'sim_0001',
 'pos': array([[ 0.16996753,  0.4720435 , -0.33467695],
        [ 0.1380482 ,  0.43700972, -0.3187462 ],
        [ 0.23911871,  0.48420787, -0.32630178],
        ...,
        [-0.8697656 ,  1.4208095 ,  0.30921957],
        [-0.7810967 ,  1.6185013 ,  0.38375667],
        [-0.80513614,  1.5754926 ,  0.39226282]], dtype=float32),
 'vel': array([[ 0.67898417,  0.15011092,  1.7421563 ],
        [ 0.67898417,  0.15011092,  1.7421563 ],
        [ 0.67898417,  0.15011092,  1.7421563 ],
        ...,
        [-0.74630326,  0.18018915,  0.76929045],
        [-0.74630326,  0.18018915,  0.76929045],
        [-0.74630326,  0.18018915,  0.76929045]], dtype=float32),
 'm': array([0.125, 0.125, 0.125, ..., 0.125, 0.125, 0.125], dtype=float32),
 'viscosity': array([0.01, 0.01, 0.01, ..., 0.01, 0.01, 0.01], dtype=float32)}

In [14]:
content[0]['m']

array([[0.1308997],
       [0.1308997],
       [0.1308997],
       ...,
       [0.1308997],
       [0.1308997],
       [0.1308997]], dtype=float32)

In [19]:
from tqdm import tqdm
from time import sleep
for x in tqdm(["a", "b", "c"]):
    sleep(0.5)

100%|██████████| 3/3 [00:01<00:00,  1.96it/s]


In [15]:
from importlib import reload
import utils.train_helper
from utils.train_helper import *
reload(utils.train_helper); from utils.train_helper import *
prepocess_dataset_files('datasets/data/dam_break_preprocessed/train', type='temp_grad', include_box=False, device = 'cuda')

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [07:40<00:00,  2.17it/s]


In [27]:
filepath = 'datasets/data/dam_break_preprocessed/train/1000_01.msgpack.zst'

decompressor = zstd.ZstdDecompressor()
with open(filepath, 'rb') as f:
    content = msgpack.unpackb(decompressor.decompress(f.read()), raw=False)

content[1].keys()

dict_keys(['pos', 'vel', 'm', 'viscosity', 'box', 'box_normals', 'density', 'temporal_density_gradient'])

In [2]:
%cd ..

/home/jakob/Bachelorarbeit/GNNDensityGradients


In [2]:
from importlib import reload
import utils.train_helper
from utils.train_helper import *
reload(utils.train_helper); from utils.train_helper import *

from datasets.density_data_module import DensityDataModule
dm = DensityDataModule(data_dir='datasets/data/dam_break_preprocessed/train', batch_size=2, data_split=(1, 0, 0), num_workers=0, shuffle=False, cache=True, device='cpu')


In [3]:
dm.setup('fit')
next(iter(dm.train_dataloader()))[0]

Setting up data module for stage  fit
Dataset is already preprepared. Not applying transform_once. (This means density & grads aren't recalculated.)
Loading dataset into memory...


100%|██████████| 1000/1000 [00:09<00:00, 108.90it/s]

Done loading dataset into memory



/home/jakob/anaconda3/envs/o3d_test2/lib/python3.10/site-packages/torch/utils/data/dataset.py:342: UserWarning: Length of split at index 1 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "
/home/jakob/anaconda3/envs/o3d_test2/lib/python3.10/site-packages/torch/utils/data/dataset.py:342: UserWarning: Length of split at index 2 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "


{'pos': tensor([[0.3768, 0.0108, 0.1589],
         [0.3721, 0.0109, 0.2014],
         [0.3586, 0.0096, 0.2570],
         ...,
         [0.9094, 1.0453, 0.6019],
         [0.9092, 1.0431, 0.6432],
         [0.9074, 1.0420, 0.6874]]),
 'vel': tensor([[-0.2274,  0.0024, -0.2001],
         [-0.2471,  0.0045, -0.1504],
         [-0.5388,  0.0049, -0.0678],
         ...,
         [-0.1252, -1.2596, -0.0718],
         [-0.1048, -1.3163, -0.1564],
         [-0.1259, -1.3333, -0.1548]]),
 'm': tensor([[0.1309],
         [0.1309],
         [0.1309],
         ...,
         [0.1309],
         [0.1309],
         [0.1309]]),
 'viscosity': tensor([[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]),
 'box': tensor([[1.4545, 1.5326, 0.7048],
         [1.4545, 1.3750, 0.5074],
         [1.4545, 1.4482, 0.0686],
         ...,
         [0.7983, 0.9205, 0.0000],
         [1.0837, 0.5266, 0.0000],
         [0.0523, 1.2732, 0.7273]]),
 'box_normals': tensor([[ 1.0

In [4]:
next(iter(dm.train_dataloader()))[0].keys()

dict_keys(['pos', 'vel', 'm', 'viscosity', 'box', 'box_normals', 'density', 'temporal_density_gradient'])

In [6]:
next(iter(dm.train_dataloader()))[0].keys()

dict_keys(['pos', 'vel', 'm', 'viscosity', 'box', 'box_normals', 'density', 'temporal_density_gradient'])

Test if cconv correctly processes params

In [1]:
hparams = {
    # Search space ---------------------------------------------

    # General
    "learning_rate" : 1e-3, # Default is 1e-3
    "batch_size"    : 2,

    # CConv architecture
    "kernel_size"              : 4,
    "num_hidden_layers"        : 2,
    "input_layer_out_channels" : 64, # Default is 32
    "hidden_units"             : 64, # Default is 64

    # CConv operation parameters
    "intermediate_activation_fn"        : "relu",
    "interpolation"                     : "linear",
    "align_corners"                     : True,
    "normalize"                         : False,
    "window_function"                   : "poly6",
    "coordinate_mapping"                : "ball_to_cube_volume_preserving",
    "filter_extent"                     : 0.025 * 6 * 1.5,
    "radius_search_ignore_query_points" : False,

    # Static parameters -----------------------------------------
    "out_units" : 1, # 1 for temporal density gradient, 3 for spatial density gradient

    # Dataset
    'dataset_dir' : 'datasets/data/dpi_dam_break/train',
    'data_split'  : (0.7, 0.15, 0.15),
    'shuffle'     : True,
    'cache'       : False,                               # Preprocess and preload dataset into memory
    'device'      : 'cuda'
}

In [3]:
from models.cconv import CConvModel
model = CConvModel(hparams)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
sum(p.numel() for p in model.parameters()), sum(p.numel() for p in model.parameters() if p.requires_grad)

(1102658, 1102658)

In [26]:
batch[0].keys()

dict_keys(['pos', 'vel', 'm', 'viscosity', 'box', 'box_normals'])

In [9]:
batch = next(iter(dm.train_dataloader()))
model(batch[0])


tensor([[-0.0318],
        [-0.0444],
        [-0.0265],
        ...,
        [-0.1553],
        [-0.1679],
        [-0.1254]], grad_fn=<MulBackward0>)

In [ ]:
model(batch)

In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
3.3 * 0.05

0.165

In [7]:
"leaky_relu".lower()

'leaky_relu'

In [11]:
0.0028473 - 2.8e-3

4.730000000000012e-05

In [14]:
import torch
import open3d.ml.torch as ml3d

inp_positions = torch.randn([20,3])
inp_features = torch.randn([20,8])
out_positions = torch.randn([20,3])

conv = ml3d.layers.ContinuousConv(in_channels=8, filters=16, kernel_size=[3,3,3], use_dense_layer_for_center = True)
out_features = conv(inp_features, inp_positions, out_positions, extents=2.0)

In [6]:
from ray.tune.logger import TBXLoggerCallback

In [11]:
class MyLoggerCallback(TBXLoggerCallback):
    def on_trial_start(self, trial_runner):
        super().on_trial_start(trial_runner)
        print('Trial started')

MyLoggerCallback().setup('test', 'test', 'test')